In [ ]:
%pip install datasets huggingface-hub
!pip uninstall -y preprocessor
!pip uninstall -y tweet-preprocessor
!pip install tweet-preprocessor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
from bs4 import BeautifulSoup,XMLParsedAsHTMLWarning
import warnings
import datasets
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer,DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, TFBertModel, TFAutoModel



In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import preprocessor as p
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [ ]:
# df=pd.read_csv("data.csv")
train=pd.read_csv("train.csv")
test=pd.read_csv("valid.csv")
df=pd.concat([train,test])
df.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [ ]:
df = df.drop(['Id', 'Tags', 'CreationDate'], axis=1)
df['concat_text'] = df['Title'] + ' ' + df['Body']
df = df.drop(['Title', 'Body'], axis=1)
df.head()

,Y,concat_text
0,LQ_CLOSE,Java: Repeat Task Every Random Seconds <p>I'm ...
1,HQ,Why are Java Optionals immutable? <p>I'd like ...
2,HQ,Text Overlay Image with Darkened Opacity React...
3,HQ,Why ternary operator in swift is so picky? <p>...
4,HQ,hide/show fab with scale animation <p>I'm usin...


In [ ]:
MAX_LEN=300

In [ ]:
def cleanse(concat_text):
    concat_text = concat_text.lower()
    concat_text = re.sub(r'[^(a-zA-Z)\s]','', concat_text)
    concat_text = [word for word in concat_text.split() if word not in stopwords.words('english')]
    sentence = []
    for word in concat_text:
        lemmatizer = WordNetLemmatizer()
        word = p.clean(word)
        sentence.append(lemmatizer.lemmatize(word, 'v'))
    return ' '.join(sentence)
df['cleaned_text'] = df['concat_text'].apply(lambda x:cleanse(x[:MAX_LEN]))
df.drop('concat_text', axis=1, inplace=True)

In [ ]:
df.head()

,Y,cleaned_text
0,LQ_CLOSE,java repeat task every random second pim alrea...
1,HQ,java optionals immutable pid like understand j...
2,HQ,text overlay image darken opacity react native...
3,HQ,ternary operator swift picky pthe question sim...
4,HQ,hideshow fab scale animation pim use custom fl...


In [ ]:
# df.to_csv("df_preprocessed.csv",index=False)
df=pd.read_csv("df_preprocessed.csv")

In [ ]:
model_path = "google-bert/bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_path)
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
id2label={0:'LQ_CLOSE',1:'HQ',2:'LQ_EDIT'}
label2id={'LQ_CLOSE':0,'HQ':1,'LQ_EDIT':2}
model=AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=3,id2label=id2label,label2id=label2id).to("cuda")
# model=TFBertModel.from_pretrained(model_path)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, param in model.base_model.named_parameters():

      param.requires_grad = True


In [ ]:
def preprocess_texts(examples):
    tokens = tokenizer(
        examples["cleaned_text"], truncation=True, padding="max_length"
    )
    tokens["labels"] = [label2id[label] for label in examples["Y"]]
    return tokens
dataset = datasets.Dataset.from_pandas(df)
tokenized_data = dataset.map(preprocess_texts, batched=True, remove_columns=["cleaned_text", "Y"])


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [ ]:
split_dataset = tokenized_data.train_test_split(test_size=0.2, seed=42)
tokenized_data = datasets.DatasetDict(split_dataset)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
lr = 5e-5
batch_size = 8
num_epochs = 2

In [ ]:
training_args = TrainingArguments(
    output_dir="stackoverflow-prediction",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


<ipython-input-15-9e3d4c5b8ae1>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)

print(f"Trainable Parameters: {trainable_params}")
print(f"Non-Trainable Parameters: {non_trainable_params}")
print(f"Total Parameters: {trainable_params + non_trainable_params}")


Trainable Parameters: 109484547
Non-Trainable Parameters: 0
Total Parameters: 109484547


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: parthpassi (parthpassi-bennett-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.381400,0.292758,0.906083,0.907236,0.906083,0.905994
2,0.251700,0.326467,0.914917,0.915037,0.914917,0.914975


TrainOutput(global_step=12000, training_loss=0.31658365885416667, metrics={'train_runtime': 9866.4215, 'train_samples_per_second': 9.73, 'train_steps_per_second': 1.216, 'total_flos': 2.5258888101888e+16, 'train_loss': 0.31658365885416667, 'epoch': 2.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.39502644538879395, 'eval_accuracy': 0.878, 'eval_precision': 0.8835090919220675, 'eval_recall': 0.878, 'eval_f1': 0.8783062025014148, 'eval_runtime': 316.7899, 'eval_samples_per_second': 37.88, 'eval_steps_per_second': 4.735, 'epoch': 1.0}


In [ ]:
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")

print("Model and tokenizer saved to 'saved_model/'")

Model and tokenizer saved to 'saved_model/'


In [ ]:
model.push_to_hub("krimson1/bert-base-stackoverflow-prediction")
tokenizer.push_to_hub("krimson1/bert-base-stackoverflow-prediction")

print("Model successfully uploaded to Hugging Face!")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Model successfully uploaded to Hugging Face!
